In [5]:
from pyhealth.datasets import MIMIC4Dataset
if __name__ == "__main__":
    dataset = MIMIC4Dataset(
        ehr_root="/srv/local/data/physionet.org/files/mimiciv/2.2/",
        ehr_tables=["diagnoses_icd", "procedures_icd", "labevents", "prescriptions"],
        dev=True,
        cache_dir="../benchmark_cache/mimic4_ehr_with_prescr/"
    )

    from pyhealth.tasks import InHospitalMortalityMIMIC4

    task = InHospitalMortalityMIMIC4()
    samples = dataset.set_task(task, num_workers=4, cache_dir="../benchmark_cache/mimic4_ihm_w_pre2/")

    from pyhealth.datasets import split_by_sample


    train_dataset, val_dataset, test_dataset = split_by_sample(
        dataset=samples, ratios=[0.7, 0.1, 0.2]
    )




Memory usage Starting MIMIC4Dataset init: 3723.0 MB
Initializing mimic4 dataset from /srv/local/data/physionet.org/files/mimiciv/2.2/|None|None (dev mode: True)
Initializing MIMIC4EHRDataset with tables: ['diagnoses_icd', 'procedures_icd', 'labevents', 'prescriptions'] (dev mode: True)
Using default EHR config: /home/johnwu3/projects/PyHealth_Branch_Testing/PyHealth/pyhealth/datasets/configs/mimic4_ehr.yaml
Memory usage Before initializing mimic4_ehr: 3723.0 MB
Initializing mimic4_ehr dataset from /srv/local/data/physionet.org/files/mimiciv/2.2/ (dev mode: True)
Memory usage After initializing mimic4_ehr: 3723.0 MB
Memory usage After EHR dataset initialization: 3723.0 MB
Memory usage Completed MIMIC4Dataset init: 3723.0 MB
Setting task InHospitalMortalityMIMIC4 for mimic4 base dataset...
Applying task transformations on data...


100%|██████████| 1000/1000 [00:31<00:00, 31.69it/s]


Indexing progress:   0%|          | 0/1 [00:00<?, ?step/s]


Fitting processors on the dataset...
Label mortality vocab: {0: 0, 1: 1}
Processing samples and saving to ../benchmark_cache/mimic4_ihm_w_pre2...
Create an account on https://lightning.ai/ to optimize your data faster using multiple nodes and large machines.
Setting multiprocessing start_method to fork. Tip: Libraries relying on lock can hang with `fork`. To use `spawn` in notebooks, move your code to files and import it within the notebook.
Storing the files under /home/johnwu3/projects/PyHealth_Branch_Testing/PyHealth/benchmark_cache/mimic4_ihm_w_pre2
Setup started with fast_dev_run=False.
Setup finished in 0.008 seconds. Found 1824 items to process.
Starting 4 workers with 1824 items. The progress bar is only updated when a worker finishes.
Workers are ready ! Starting data processing...


Progress:   0%|          | 0/1824 [00:00<?, ?it/s]

Rank 0 inferred the following `['str', 'str', 'tensor', 'no_header_tensor:1']` data format.
Rank 2 inferred the following `['str', 'str', 'tensor', 'no_header_tensor:1']` data format.
Rank 1 inferred the following `['str', 'str', 'tensor', 'no_header_tensor:1']` data format.
Rank 3 inferred the following `['str', 'str', 'tensor', 'no_header_tensor:1']` data format.
Worker 0 is terminating.
Worker 2 is terminating.
Worker 1 is terminating.
Worker 3 is terminating.
Worker 2 is done.
Worker 1 is done.
Worker 0 is done.
Worker 3 is done.
Workers are finished.
Finished data processing!
Cached processed samples to ../benchmark_cache/mimic4_ihm_w_pre2


In [4]:
len(samples)

912

In [6]:
dataset.get_patient(dataset.unique_patient_ids[0]).get_events()

Found 1000 unique patient IDs


[Event(event_type='patients', timestamp=datetime.datetime(2025, 12, 18, 12, 2, 11, 324654), attr_dict={'gender': 'M', 'anchor_age': '73', 'anchor_year': '2137', 'anchor_year_group': '2008 - 2010', 'dod': None}),
 Event(event_type='labevents', timestamp=datetime.datetime(2136, 6, 21, 12, 25), attr_dict={'itemid': '51006', 'label': 'Urea Nitrogen', 'fluid': 'Blood', 'category': 'Chemistry', 'value': '16', 'valuenum': '16', 'valueuom': 'mg/dL', 'flag': None, 'storetime': '2136-06-21 17:20:00'}),
 Event(event_type='labevents', timestamp=datetime.datetime(2136, 6, 21, 12, 25), attr_dict={'itemid': '50983', 'label': 'Sodium', 'fluid': 'Blood', 'category': 'Chemistry', 'value': '144', 'valuenum': '144', 'valueuom': 'mEq/L', 'flag': None, 'storetime': '2136-06-21 17:20:00'}),
 Event(event_type='labevents', timestamp=datetime.datetime(2136, 6, 21, 12, 25), attr_dict={'itemid': '50974', 'label': 'Prostate Specific Antigen', 'fluid': 'Blood', 'category': 'Chemistry', 'value': '___', 'valuenum': '

In [7]:
samples[0]

{'patient_id': '11753938',
 'admission_id': '29579516',
 'labs': tensor([[  0.0000,   0.0000, 141.0000,   0.0000,   0.0000,   0.0000,   4.5000,
            0.0000,   0.0000,   0.0000, 102.0000,   0.0000,   0.0000,   0.0000,
            0.0000,   0.0000, 162.0000,   0.0000,   0.0000,   0.0000,   2.2000,
           18.0000,   0.0000,   0.0000,   0.0000,   0.0000,   5.2000]]),
 'mortality': tensor([0.])}

In [3]:
from pyhealth.datasets import get_dataloader
train_dataloader = get_dataloader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = get_dataloader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = get_dataloader(test_dataset, batch_size=32, shuffle=False)

from pyhealth.models import RNN

model = RNN(
    dataset=samples,
)

from pyhealth.trainer import Trainer

trainer = Trainer(model=model, metrics=["roc_auc"])
print(trainer.evaluate(test_dataloader))

trainer.train(
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    epochs=10,
    monitor="roc_auc",  # Monitor roc_auc specifically
    optimizer_params={"lr": 1e-4}  # Using learning rate of 1e-4
)


/home/johnwu3/projects/PyHealth_Branch_Testing/PyHealth/pyhealth/sampler/sage_sampler.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/home/johnwu3/projects/PyHealth_Branch_Testing/PyHealth/pyhealth/metrics/calibration.py:122: SyntaxWarning: invalid escape sequence '\c'
  accuracy of 1. Thus, the ECE is :math:`\\frac{1}{3} \cdot 0.49 + \\frac{2}{3}\cdot 0.3=0.3633`.


RNN(
  (embedding_model): EmbeddingModel(embedding_layers=ModuleDict(
    (labs): Linear(in_features=27, out_features=128, bias=True)
  ))
  (rnn): ModuleDict(
    (labs): RNNLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (rnn): GRU(128, 128, batch_first=True)
    )
  )
  (fc): Linear(in_features=128, out_features=1, bias=True)
)
Metrics: ['roc_auc']
Device: cuda



Evaluation: 100%|██████████| 6/6 [00:02<00:00,  2.31it/s]

{'roc_auc': 0.8403575989782885, 'loss': 0.6699792544047037}
Training:
Batch size: 32
Optimizer: <class 'torch.optim.adam.Adam'>
Optimizer params: {'lr': 0.0001}
Weight decay: 0.0
Max grad norm: None
Val dataloader: <torch.utils.data.dataloader.DataLoader object at 0x7fadb9a964b0>
Monitor: roc_auc
Monitor criterion: max
Epochs: 10
Patience: None



Epoch 0 / 10:   0%|          | 0/20 [00:00<?, ?it/s]

...

--- Eval epoch-8, step-180 ---
roc_auc: 0.7111
loss: 0.0852



Epoch 9 / 10:   0%|          | 0/20 [00:00<?, ?it/s]

--- Train epoch-9, step-200 ---
loss: 0.1144


Evaluation: 100%|██████████| 3/3 [00:00<00:00, 97.20it/s]

--- Eval epoch-9, step-200 ---
roc_auc: 0.7111
loss: 0.0839
Loaded best model
